In [1]:
import sqlite3,pathlib,os
from Essential_Functions import Spacing,CalculateSpacing,ConnectDatabase,GetTableInfo

import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Function to get all PlayerStatistics
def GetPlayerStatistics(cr,db):
    print("Getting PlayerStatistics...")
    Spacing(1)
    cr.execute("""SELECT * FROM PlayerStatistics""")
    PlayerStatistics = cr.fetchall()
    return PlayerStatistics

In [3]:
# Function to get PlayerStatistics table headings
def GetHeadings(cr,db):
    TableInfo = GetTableInfo(cr,db,"PlayerStatistics")
    Headings = []
    for ColumnInfo in TableInfo:
        Headings.append(ColumnInfo[1])
    return Headings

In [4]:
cr,db = ConnectDatabase("WoTData.db")

Connected to WoTData.db Successfully.



In [5]:
Headings = GetHeadings(cr,db)
Headings

['account_id',
 'last_battle_time',
 'created_at',
 'updated_at',
 'global_rating',
 'clan_id',
 'statistics$all$spotted',
 'statistics$all$battles_on_stunning_vehicles',
 'statistics$all$max_xp',
 'statistics$all$avg_damage_blocked',
 'statistics$all$direct_hits_received',
 'statistics$all$explosion_hits',
 'statistics$all$piercings_received',
 'statistics$all$piercings',
 'statistics$all$max_damage_tank_id',
 'statistics$all$xp',
 'statistics$all$survived_battles',
 'statistics$all$dropped_capture_points',
 'statistics$all$hits_percents',
 'statistics$all$draws',
 'statistics$all$max_xp_tank_id',
 'statistics$all$battles',
 'statistics$all$damage_received',
 'statistics$all$avg_damage_assisted',
 'statistics$all$max_frags_tank_id',
 'statistics$all$avg_damage_assisted_track',
 'statistics$all$frags',
 'statistics$all$stun_number',
 'statistics$all$avg_damage_assisted_radio',
 'statistics$all$capture_points',
 'statistics$all$stun_assisted_damage',
 'statistics$all$max_damage',
 'stat

In [6]:
PlayerStatistics = GetPlayerStatistics(cr,db)
# Convert the list of player statistics into a pandas dataframe
PlayerStatistics_DataFrame = pd.DataFrame(PlayerStatistics, columns = Headings)
PlayerStatistics_DataFrame

Getting PlayerStatistics...



,account_id,last_battle_time,created_at,updated_at,global_rating,clan_id,statistics$all$spotted,statistics$all$battles_on_stunning_vehicles,statistics$all$max_xp,statistics$all$avg_damage_blocked,...,statistics$all$damage_dealt,statistics$all$no_damage_direct_hits_received,statistics$all$max_frags,statistics$all$shots,statistics$all$explosion_hits_received,statistics$all$tanking_factor,statistics$trees_cut,logout_at,vehicle_personality_preference,pca_rating
0,500000050,1306253950,1302642174,1639778437,666,None,109,0,405,0.00,...,17457,0,3,731,0,0.00,0,1306253730,0.8888888888888888$1.0$0.6111111111111109$0.94...,-4.068629
1,500000052,1570480020,1302642639,1640351593,4870,None,16080,0,2574,78.96,...,7022850,1944,11,79020,99,0.23,801,1575054990,0.013144590495449946$0.08998988877654193$0.243...,1.736649
2,500000058,1629644007,1302624312,1639778442,8352,500000449,24620,65,2730,227.91,...,30590026,7005,11,240683,2544,0.29,11503,1629644035,0.0$0.021239954075774968$0.043340987370838115$...,8.389127
3,500000087,1516303363,1302645029,1639778447,3414,None,2818,0,1996,228.16,...,1063472,3265,9,26156,184,0.47,2528,1516303357,0.0334128878281623$0.17899761336515513$0.26491...,-0.553388
4,500000098,1313081476,1302889690,1639778450,3409,500000024,1900,0,1956,0.00,...,874819,0,7,12581,0,0.00,0,1313081579,0.021052631578947364$0.07368421052631578$0.147...,-2.405266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446855,504999989,1415527528,1347380647,1639957393,789,None,281,0,754,22.50,...,42146,2,4,2494,0,0.12,0,1607175693,0.06329113924050633$0.17721518987341772$0.5189...,NaN
1446856,504999991,1621116468,1347380649,1639957395,5616,None,16091,30,2755,492.32,...,16373913,22496,11,170288,2367,0.47,12355,1621116290,0.0$0.0506161971830986$0.09286971830985917$0.3...,NaN
1446857,504999993,1641681878,1347380650,1641681879,7800,None,24096,387,2601,559.37,...,30428222,35005,9,230683,8839,0.50,21532,1641681599,0.0$0.020393811533052034$0.07278481012658226$0...,NaN
1446858,504999995,1601832844,1347380654,1639957398,1216,None,574,0,656,96.58,...,122818,32,5,14453,12,0.17,15,1601832503,0.13218390804597702$0.27586206896551724$0.3333...,NaN


In [7]:
AccountID_DataFrame = PlayerStatistics_DataFrame["account_id"]
# Drop irrelevant columns from the dataframe which are not used for PCA
PlayerStatistics_DataFrame.drop(["account_id", "clan_id", "statistics$all$max_damage_tank_id", "statistics$all$max_xp_tank_id", "statistics$all$max_frags_tank_id", "vehicle_personality_preference", "pca_rating"], axis = 1, inplace = True)
PlayerStatistics_DataFrame

,last_battle_time,created_at,updated_at,global_rating,statistics$all$spotted,statistics$all$battles_on_stunning_vehicles,statistics$all$max_xp,statistics$all$avg_damage_blocked,statistics$all$direct_hits_received,statistics$all$explosion_hits,...,statistics$all$wins,statistics$all$losses,statistics$all$damage_dealt,statistics$all$no_damage_direct_hits_received,statistics$all$max_frags,statistics$all$shots,statistics$all$explosion_hits_received,statistics$all$tanking_factor,statistics$trees_cut,logout_at
0,1306253950,1302642174,1639778437,666,109,0,405,0.00,0,0,...,62,75,17457,0,3,731,0,0.00,0,1306253730
1,1570480020,1302642639,1640351593,4870,16080,0,2574,78.96,6290,31,...,4636,3969,7022850,1944,11,79020,99,0.23,801,1575054990
2,1629644007,1302624312,1639778442,8352,24620,65,2730,227.91,29762,2943,...,12575,9158,30590026,7005,11,240683,2544,0.29,11503,1629644035
3,1516303363,1302645029,1639778447,3414,2818,0,1996,228.16,10512,164,...,1292,1304,1063472,3265,9,26156,184,0.47,2528,1516303357
4,1313081476,1302889690,1639778450,3409,1900,0,1956,0.00,0,0,...,793,659,874819,0,7,12581,0,0.00,0,1313081579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446855,1415527528,1347380647,1639957393,789,281,0,754,22.50,9,0,...,168,189,42146,2,4,2494,0,0.12,0,1607175693
1446856,1621116468,1347380649,1639957395,5616,16091,30,2755,492.32,68875,1265,...,8304,8219,16373913,22496,11,170288,2367,0.47,12355,1621116290
1446857,1641681878,1347380650,1641681879,7800,24096,387,2601,559.37,104123,4601,...,11495,9671,30428222,35005,9,230683,8839,0.50,21532,1641681599
1446858,1601832844,1347380654,1639957398,1216,574,0,656,96.58,131,1,...,494,528,122818,32,5,14453,12,0.17,15,1601832503


In [8]:
# Scaling the PlayerStatistics dataframe
SS = StandardScaler()
PlayerStatistics_Scaled = SS.fit_transform(PlayerStatistics_DataFrame)

# Performing a Principle Component Analysis on the scaled PlayerStatistics values
PrincipleComponentAnalysis = PCA (n_components = 1)
Components = PrincipleComponentAnalysis.fit_transform(PlayerStatistics_Scaled)

# Converting the PCA results into a pandas dataframe for ease of use
PlayerStatistics_PCA = pd.DataFrame(data = Components, columns = ["pca_rating"])

# Analysing the dataframe to ensure it has worked
PlayerStatistics_PCA

,pca_rating
0,-4.068629
1,1.736649
2,8.389127
3,-0.553388
4,-2.405266
...,...
1446855,-3.344497
1446856,6.207880
1446857,10.290081
1446858,-2.908861


In [9]:
# Combining the player IDs from the player statistics dataframe with the calculated PCA results
PlayerStatistics_PCA = pd.concat([AccountID_DataFrame,PlayerStatistics_PCA], axis = 1)
PlayerStatistics_PCA

,account_id,pca_rating
0,500000050,-4.068629
1,500000052,1.736649
2,500000058,8.389127
3,500000087,-0.553388
4,500000098,-2.405266
...,...,...
1446855,504999989,-3.344497
1446856,504999991,6.207880
1446857,504999993,10.290081
1446858,504999995,-2.908861


In [10]:
# Get the first player ID in the dataframe as the test ID, also then get their PCA result
Test_PlayerID = int(PlayerStatistics_PCA.iloc[0][0])
Test_PlayerPCA = float(PlayerStatistics_PCA.iloc[0][1])

# Print the test player's PCA data
print(str(Test_PlayerID), str(type(Test_PlayerID)))
print(str(Test_PlayerPCA), str(type(Test_PlayerPCA)))
Spacing(1)

Total = len(PlayerStatistics)
# Loop through all players in the player statistics dataframe by position
for Position in range(Total):
    PlayerID = int(PlayerStatistics_PCA.iloc[Position][0])
    PlayerPCA = float(PlayerStatistics_PCA.iloc[Position][1])
    
    # Update the PCA value of the current player with their PCA result in the player statistics dataframe
    cr.execute("""UPDATE PlayerStatistics SET pca_rating = ? WHERE account_id = ?""", (PlayerPCA, PlayerID))
    # Output the current progress of the process
    if Position % 10000 == 0:
        print("PCA Progress: " + str(round((Position / Total) * 100, 5)) + "%")
# Commit the changes   
db.commit()
        
print("PCA Complete.")

500000050 <class 'int'>
-4.068628880554472 <class 'float'>

PCA Progress: 0.0%
PCA Progress: 0.69115%
PCA Progress: 1.3823%
PCA Progress: 2.07346%
PCA Progress: 2.76461%
PCA Progress: 3.45576%
PCA Progress: 4.14691%
PCA Progress: 4.83806%
PCA Progress: 5.52921%
PCA Progress: 6.22037%
PCA Progress: 6.91152%
PCA Progress: 7.60267%
PCA Progress: 8.29382%
PCA Progress: 8.98497%
PCA Progress: 9.67613%
PCA Progress: 10.36728%
PCA Progress: 11.05843%
PCA Progress: 11.74958%
PCA Progress: 12.44073%
PCA Progress: 13.13189%
PCA Progress: 13.82304%
PCA Progress: 14.51419%
PCA Progress: 15.20534%
PCA Progress: 15.89649%
PCA Progress: 16.58764%
PCA Progress: 17.2788%
PCA Progress: 17.96995%
PCA Progress: 18.6611%
PCA Progress: 19.35225%
PCA Progress: 20.0434%
PCA Progress: 20.73456%
PCA Progress: 21.42571%
PCA Progress: 22.11686%
PCA Progress: 22.80801%
PCA Progress: 23.49916%
PCA Progress: 24.19032%
PCA Progress: 24.88147%
PCA Progress: 25.57262%
PCA Progress: 26.26377%
PCA Progress: 26.95492%
PCA